<a href="https://colab.research.google.com/github/jvrscak/Analiza-utjecaja-prehrane-na-rezultate-sporta-a/blob/main/projekt_iz_PZAP_JanVr%C5%A1%C4%8Dak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROJETK PZAP**  
**ANALIZA UTJECAJA PREHRANE NA REZULTATE SPORTAŠA**  
**STUDIJ: INFORMACIJSKI I POSLOVNI SUSTAVI**  
**SMJER: UMJETNA INTELIGENCIJA U POSLOVANJU**  
**GODINA: 3.**  
**IME I PREZIME: JAN VRŠČAK**  
**VARAŽDIN, SIJEČANJ, 2026.**


**UVOD**  
Cilj projekta je ispitati postoji li povezanost između nutritivnog sastava prehrane npr. kalorije, proteini, ugljikohidrati te sportskih performansi.  
Prehrana ima važnu ulogu u životima sportaša jer izravno utječe na fizičku spremu, oporavak i postizanje sportskih rezultata. U suvremenom sportu sve se veći naglasak stavlja na individualizirani pristup prehrani sportaša, a odluke se temelje na analizi podataka, a ne prema preporukama. Osobna motivacija za ovu temu je moja ljubav prema sportu te bavljenje sportom kroz cijeli život. Osim toga, motivacija za odabir ove teme je da se povežu sport i analiza podataka i da se kroz konkretan primjer pokaže kako se podatci iz različitih izvora mogu integrirati i analizirati u svrhu donošenja zaključaka. Korištenjem podataka o nutritivnim vrijednostima namirnica i podataka o sportašima moguće je istražiti potencijalne povezanosti između prehrambenih karakteristika i sportskih performansi. Dodatna motivacija je tehnički aspekt da se povežu heterogeni skupovi podataka, JSON i CSV, te njihovo pretprocesiranje i integracija te implementacija jednostavnog sustava za pohranu i pristup podatcima.

**OPIS IMPLEMENTACIJE**

**Uvođenje i učitavanje podataka**



U projektu korištena su dva odvojena skupa podataka. Prvi skup je CSV format koji se odnosi na sportaše te njihove namirnice koej koriste i sportski uspjeh. Taj skup podataka je generiran preko web stranice generatedata.com.[2] Drugi skup podataka je u JSON formatu i odnosi se na nutritivne vrijednosti namirnica, uključujući energetsku vrijednost, proteine, ugljikohidrate, masti. Taj format je preuzet sa stranica OpenFoodFacts.[1]   
Kako bih preuzeo JSON podatke sa stranice OpenFoodFacts, morao sam preuzeti podatke preko sljedećeg linka: https://world.openfoodfacts.org/api/v0/product/4056489148739.json , a to znači da sam prvo pronašao proizvod preko stranice OpenFoodFacts i onda sam upisao barkod tog proizvoda u link i preuzeo JSON podatke.[1]

In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
df_sport = pd.read_csv('podaci_o_sportasu.csv')
df_sport.head()

,athlete_id,sport,age,weight_kg,height_cm,training_hours_per_week,product_1,product_2,product_3,performance_score
0,581,Running,39,69,172,19,Shrimp,Cottage Cheese,Walnuts,87
1,567,Running,40,85,197,18,Tomato,Blueberries,Apple,53
2,2,Football,27,61,163,5,Blueberries,Shrimp,Salmon,95
3,235,Cycling,37,65,193,16,Sunflower Seeds,Apple,Broccoli,67
4,571,Cycling,18,63,178,10,Oatmeal,Brown Rice,Walnuts,78


In [6]:
with open("namirnice.json", "r") as f:
    namirnice = json.load(f)

In [7]:
type(namirnice)
namirnice[0]

{'product_name': 'Eggs',
 'nutriments': {'energy-kcal_value': 78.65,
  'proteins_value': 7.755,
  'carbohydrates_value': 0,
  'sugars_value': 0,
  'fat_value': 5.28,
  'saturated-fat_value': 1.485,
  'fiber_value': 0,
  'calcium_value': 0,
  'potassium_value': 0}}

In [8]:
type(namirnice)
namirnice[3]

{'product_name': 'Almonds',
 'nutriments': {'energy-kcal_value': 186,
  'proteins_value': 7.4,
  'carbohydrates_value': 1.4,
  'sugars_value': 1,
  'fat_value': 16,
  'saturated-fat_value': 1.3,
  'fiber_value': 2.4,
  'calcium_value': 0.236,
  'potassium_value': 0.659}}

**Pretprocesiranje podataka**  

Pretprocesiranje je provedeno s ciljem pripreme podataka za daljnu analizu i integraciju. Treba se osigurati konzistentnost i ispravnost podataka budući da su skupovi u različitim formatima.

Za početak, uklonio sam nepotrebne nutrijente i ostavio sam energetsku vrijednost, proteine, ugljikohidrate i masti. Napravio sam tablični oblik i izravnao ugniježđeni JSON.

In [14]:
hrana = []

for i in namirnice:
    namirnice = i.get("nutriments", {})
    hrana.append({
        "product_name": i.get("product_name", "").lower().strip(),
        "calories": i.get("nutriments", {}).get("energy-kcal_value", 0),
        "proteins": i.get("nutriments", {}).get("proteins_value", 0),
        "carbohydrates": i.get("nutriments", {}).get("carbohydrates_value", 0),
        "fats": i.get("nutriments", {}).get("fat_value", 0)
    })

hrana_df = pd.DataFrame(hrana)
hrana_df.head()


,product_name,calories,proteins,carbohydrates,fats
0,eggs,78.65,7.755,0.0,5.28
1,banana,90.50,1.060,19.7,0.25
2,broccoli,40.40,2.000,7.2,0.40
3,almonds,186.00,7.400,1.4,16.00
4,olive oil,828.00,0.000,0.0,92.00


Ukoliko postoje zapisi bez imena proizvoda, potrebno ih je ukloniti i treba osigurati da svi podatci o nutritivnim vrijednostima budu numeričkog tipa.

In [11]:
hrana_df = hrana_df[hrana_df["product_name"] != ""]

In [15]:
from numpy._core import numeric
numeric_columns = ["calories", "proteins", "carbohydrates", "fats"]
hrana_df[numeric_columns] = hrana_df[numeric_columns].apply(pd.to_numeric, errors="coerce")
hrana_df.fillna(0, inplace=True)

Nakon što je odrađeno pretprocesiranje JSON podataka, sada je potrebno pretprocesirati CSV podatke. Podatci o sportašima zapisani su u CSV formatu. Tamo su nam bitni podatci o konzumiranim proizvodima (product_1, product_2, product_3) i oni služe kao poveznica za podatke o namirnicama. Nema nedostajućih vrijednosti pa nema potrebe za uklanjanjem. Ali, zato sam nazive proizvoda oba skupa standardizirao, odnosno pretvorio ih u mala slova i uklonio nepotrebne znakove kako bi se mogla provesti faza integracije.

In [16]:
for col in ["product_1", "product_2", "product_3"]:
    df_sport[col] = df_sport[col].str.lower().str.strip()

In [18]:
df_sport.head()

,athlete_id,sport,age,weight_kg,height_cm,training_hours_per_week,product_1,product_2,product_3,performance_score
0,581,Running,39,69,172,19,shrimp,cottage cheese,walnuts,87
1,567,Running,40,85,197,18,tomato,blueberries,apple,53
2,2,Football,27,61,163,5,blueberries,shrimp,salmon,95
3,235,Cycling,37,65,193,16,sunflower seeds,apple,broccoli,67
4,571,Cycling,18,63,178,10,oatmeal,brown rice,walnuts,78


**INTEGRACIJA PODATAKA**

In [19]:
def get_nutrients(product_name, hrana_df):
    row = hrana_df[hrana_df["product_name"] == product_name]
    if row.empty:
        return pd.Series({
            "calories": 0,
            "proteins": 0,
            "carbohydrates": 0,
            "fats": 0
        })

    return row.iloc[0][["calories", "proteins", "carbohydrates", "fats"]]

**LITERATURA**  
[1] OpenFoodFacts, "OpenFoodFacts - Data", OpenFoodFacts.[Online]. Dostupno: https://world.openfoodfacts.org/data . [Pristupljeno: 21.1.2026.]  
[2] GenerateData, "Generate Data - Random Data Generator", GenerateData. [Online]. Dostupno: https://generatedata.com/ . [Pristupljeno: 20.1.2026.]